In [3]:
import gradio as gr
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

import hopsworks
import joblib
import datetime
import os
import requests



In [13]:
%cd ..
from features.weather_measures import historical_weather_measures
%cd notebooks

c:\Users\Benj3\OneDrive\Dokumenter\VSCode
[WinError 2] Den angivne fil blev ikke fundet: 'notebooks'
c:\Users\Benj3\OneDrive\Dokumenter\VSCode


In [2]:
project = hopsworks.login()
fs = project.get_feature_store()

mr = project.get_model_registry()
model = mr.get_model("electricity_price_prediction_model", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/dk_electricity_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/556180
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
def predict():
    today = get_date()
    temp = get_temp(today)
    df = pd.DataFrame({"date": [today], "temperature_2m": [temp]})
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
    df['day'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    holidays = calendar().holidays(start=df['date'].min(), end=df['date'].max())
    df['holiday'] = df['date'].isin(holidays).astype(int)

    demand = model.predict(df.drop(columns=['date']))[0]
    return [today, temp, demand]


In [5]:
def get_date():
    today = datetime.datetime.today()
    return today.date()

In [16]:
def get_temp(date):
    historical_weather_measures
    return historical_weather_measures(date)['temperature_2m'].values[0]

In [17]:
demo = gr.Interface(
    fn = predict,
    title = "DK1 Electricity Price Prediction",
    description ="Daily DK1 electricity proce prediction, based on current date (day, holiday, ...) and (forecast) temperature average.",
    allow_flagging = "never",
    inputs = [],
    outputs = [
        gr.Textbox(label="Date"),
        gr.Textbox(label="Temperature forecast [℃]"),
        gr.Textbox(label="Predicted demand [MWh]"),
    ]
)

        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
